In [4]:
import numpy as np
import pandas as pd
from dataclasses import make_dataclass
import os


In [2]:
from pfs.drp.stella.readLineList import ReadLineListTask,  ReadLineListConfig
from pfs.drp.stella import DetectorMap
import lsst.daf.persistence as dafPersist

In [5]:
base, rerun = "/data/drp/sps", "ginga/detrend"

dataDir = os.path.join(base, "rerun", rerun) if rerun else base
calibRoot = "/data/drp/sps/CALIB/"

butler = dafPersist.Butler(dataDir) #, calibRoot=calibRoot)

/tmp/ipykernel_14986/1902927490.py:6: FutureWarning: Gen2 Butler has been deprecated (PfsButler). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  butler = dafPersist.Butler(dataDir) #, calibRoot=calibRoot)
/tmp/ipykernel_14986/1902927490.py:6: FutureWarning: Gen2 Butler has been deprecated (PfsMapper). It will be removed sometime after v23.0 but no earlier than the end of 2021.
  butler = dafPersist.Butler(dataDir) #, calibRoot=calibRoot)


lsst.CameraMapper INFO: Loading exposure registry from /data/drp/sps/registry.sqlite3
lsst.CameraMapper INFO: Loading calib registry from /data/drp/sps/CALIB/calibRegistry.sqlite3
lsst.CameraMapper INFO: Loading calib registry from /data/drp/sps/CALIB/calibRegistry.sqlite3


In [6]:
specId = 2
arm = "n"

In [7]:
dataId = dict(visit=45367, arm=arm, spectrograph=specId)


In [8]:
exp = butler.get('calexp', dataId)

In [9]:
pfsConfig = butler.get("pfsConfig", dataId).select(spectrograph=specId)
#fiberIds = pfsConfig.fiberId #np.array([653, 714,  843,  906,  990, 1052, 1115, 1176, 1238, 1301, 1304])
fiberIds = pfsConfig.fiberId[pfsConfig.spectrograph == specId]

In [10]:
detMap = DetectorMap.readFits("/data/drp/sps/CALIB/DETECTORMAP/pfsDetectorMap-045367-n2.fits")

In [11]:
linesConfig = ReadLineListConfig()
linesConfig.minIntensity = 5.0
linesConfig.exclusionRadius = 2

In [12]:
lines = ReadLineListTask(config=linesConfig).run(detectorMap=detMap, metadata=exp.getMetadata())

In [13]:
lines.toDataFrame()

,description,wavelength,intensity,status,transition,source
448,KrI,935.48000,100.0,8,UNKNOWN,0
450,KrI,945.34700,20.0,0,UNKNOWN,0
452,KrI,954.35100,30.0,0,UNKNOWN,0
457,KrI,969.04900,10.0,2,UNKNOWN,0
458,KrI,970.68800,50.0,2,UNKNOWN,0
459,KrI,971.75100,15.0,2,UNKNOWN,0
462,KrI,974.57800,50.0,2,UNKNOWN,0
463,KrI,975.44352,5.0,10,UNKNOWN,0
467,KrI,984.10300,5.0,2,UNKNOWN,0
468,KrI,985.89400,500.0,2,UNKNOWN,0


In [18]:
goodWaves = [1220.78754, 1182.26136, 1179.564, 1146.0614, 1119.017, 1087.788, 1036.321,  985.894,  970.688 ]
lamps = ["Kr"]

In [19]:
dfLines = lines.toDataFrame()
dfLines = dfLines[dfLines.wavelength.isin(goodWaves)]

In [20]:
tmp = list()
# format
columns = ["fiber", "wavelength", "peak", "element", "arm", "X", "Y", "lamp", "description", "status", "source"]

Point = make_dataclass("Point", [("fiber",int), ("wavelength", float), ("peak",int), ("element", str), ("arm", str),\
                                 ("X",float), ("Y",float), ("lamp", str), ("description",str), \
                                 ("status",int), ("source",str)])

for i, lin in enumerate(dfLines.iterrows()):
    for fib in fiberIds:
        info = lin[1]
        wave = info["wavelength"]
        description = info["description"]
        status = info["status"]
        source = info["source"]
        element = description
        lamp = lamps[0]
        x = detMap.findPoint(fib, wave)[0]
        y = detMap.findPoint(fib, wave)[1]
        
        
        tmp.append(Point(fib- 651*(specId-1), wave, i+1, element, arm, x, y, lamp, description, status, source))
        

In [21]:
peaks = pd.DataFrame(data=tmp)

In [22]:
peaks.fiber.unique()

array([  2,  63, 192, 255, 339, 401, 464, 525, 587, 650])

In [23]:
peaks["dcb_wheel"] = np.nan
peaks["exptime"] = np.nan

In [24]:
peaks

,fiber,wavelength,peak,element,arm,X,Y,lamp,description,status,source,dcb_wheel,exptime
0,2,970.68800,1,KrI,n,4056.959104,440.204299,Kr,KrI,2,0,NaN,NaN
1,63,970.68800,1,KrI,n,3676.739870,420.900388,Kr,KrI,2,0,NaN,NaN
2,192,970.68800,1,KrI,n,2878.421546,391.562462,Kr,KrI,2,0,NaN,NaN
3,255,970.68800,1,KrI,n,2489.794406,383.253775,Kr,KrI,2,0,NaN,NaN
4,339,970.68800,1,KrI,n,1972.153400,379.230509,Kr,KrI,2,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,401,1220.78754,9,KrI,n,1578.732988,3435.109973,Kr,KrI,0,0,NaN,NaN
86,464,1220.78754,9,KrI,n,1190.137856,3444.251214,Kr,KrI,0,0,NaN,NaN
87,525,1220.78754,9,KrI,n,813.090374,3458.506169,Kr,KrI,0,0,NaN,NaN
88,587,1220.78754,9,KrI,n,428.506861,3478.765967,Kr,KrI,0,0,NaN,NaN


In [ ]:
if False:
    peaks.to_csv("SMx_peakList.csv")